# LSTM/GRU modeling

## 1. Modeling
Used Functions
- create_dataset(): transform X dataset into dataset splitted in timestep-unit
- BlockingTimeSeriesSplit(): make train-validation split object
** reference : https://gmnam.tistory.com/230#:~:text=class%20BlockingTimeSeriesSplit%28%29%3A%20def%20__init__%28self%2C%20n_splits%29%3A%20self.n_splits%20%3D%20n_splits,indices%20%5Bstart%3A%20mid%5D%2C%20indices%20%5Bmid%20%2B%20margin%3A%20stop%5D
- make_split(): train-validation split using BlockedTimeSeriesSplit() object
- make_lstm_model(): make lstm model structure
- make_gru_model(): make gru model structure
- grid_search(): execute grid search for lstm/gru model and save the models and performance results in history_dict.pkl in the designated folder path

## 2. Model Selection
Outcomes
* df_results.pkl & df_selection.pkl
-- df_results: concatenate all candidate LSTM/GRU models result by model specification and each validation
-- df_selection: concatenate all candidate LSTM/GRU models result by model specification
* using df_selection for selecting the best model among all candidate models
  

##1. Modeling

### Set up

In [ ]:
import tensorflow as tf
from tensorflow import keras

import os
import random
import tempfile
import pickle

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import numpy as np
import pandas as pd
import seaborn as sns

import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from keras.utils.np_utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, GRU, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.metrics import AUC, Recall, Precision
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers
from keras import backend as K

In [ ]:
## import data and metadata

import os, sys
from google.colab import drive
drive.mount('/content/drive')
path='drive/MyDrive/Capstone(Team10)/code'


file = tf.keras.utils

with open(path+'/data/X_data_full.pkl','rb') as f:
  X_data=pickle.load(f)

raw_df = pd.read_csv(path+'/data/rawdata_USA.csv', index_col=0, parse_dates=True)
raw_df.index.name='date'

metadata=pd.read_csv(path+'/data/metadata_final.csv')

X_data.head(5)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,LCEAPR01,XTIMVA01,PRMNTO01,MABMM301,PRCNTO01,STMNIS01,SLWHTO02,ODMNTO02,XTEXVA01,CCRETT01,...,IRLTLT01,IR3TCD01,BSPRTE02,LREMTTTT,LRHU24TT,LRUNTTTT,BSCURT02,3MTBILL,10YT,103SPREAD
date,,,,,,,,,,,,,,,,,,,,,
1973-01-01,0.004975,0.047149,0.007775,0.009922,0.013699,-0.006349,0.040962,0.014942,0.084607,-0.011478,...,0.10,0.38,2.569360,-0.2,-1.1,-0.3,0.3890,0.34,0.10,1.05
1973-02-01,0.002478,0.044624,0.015954,0.004679,0.020691,-0.006390,0.016314,0.023197,0.022962,-0.041932,...,0.18,0.53,-6.665425,0.4,0.7,0.1,1.1154,0.19,0.18,1.04
1973-03-01,0.004938,-0.012775,0.001725,0.001473,0.014144,0.006390,0.019215,0.022260,0.046380,-0.035349,...,0.07,0.71,0.992926,0.3,-0.5,-0.1,-0.1107,0.49,0.07,0.62
1973-04-01,0.004914,-0.009876,-0.001709,0.005382,-0.027771,0.006349,0.021154,-0.008716,0.033822,0.002986,...,-0.04,0.26,-3.104780,-0.1,0.6,0.1,-0.4168,0.17,-0.04,0.41
1973-05-01,0.004890,0.062043,0.006343,0.008624,-0.010260,0.000000,0.014101,0.006697,0.012248,-0.009481,...,0.18,0.24,-13.030239,0.0,-0.3,-0.1,0.2500,0.10,0.18,0.49


### Create Dataset X split into timestep unit

* input shape = ( X=[#obs, #feature] , y=[#obs, 1], time_steps=t )
* output shape = ( [#obs-(t-1), t, #feature], [#obs-(t-1), ] )

In [ ]:
def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []

    if type(Xs)==pd.core.frame.DataFrame:
      for i in range(len(X) - time_steps + 1):
          v = X.iloc[i:(i + time_steps)].values
          Xs.append(v)
          ys.append(y.iloc[i + time_steps-1].values)
          Xs=np.array(Xs)
          ys=np.array(ys)

    else:
      for i in range(len(X) - time_steps + 1):
          v = X[i:(i + time_steps)]
          Xs.append(v)
          ys.append(y[i + time_steps-1])

    return np.array(Xs), np.array(ys)

In [ ]:
a, b=create_dataset(X_data, raw_df['y_oecd'][-600-6:-6], 24)
a.shape, b.shape

((577, 24, 45), (577,))

### Make BlockSplit object and Split function

In [ ]:
class BlockingTimeSeriesSplit():
    def __init__(self, n_splits):
        self.n_splits = n_splits

    def get_n_splits(self, groups):
        return self.n_splits

    def split(self, X, y=None, groups=None):
        n_samples = len(X)
        k_fold_size = n_samples // self.n_splits
        indices = np.arange(n_samples)

        margin = 0
        for i in range(self.n_splits):
            start = i * k_fold_size
            stop = start + k_fold_size
            mid = int(0.8 * (stop - start)) + start
            yield indices[start: mid], indices[mid + margin: stop]


In [ ]:
cmap_data = plt.cm.Paired
cmap_cv = plt.cm.coolwarm
plt.style.use('fivethirtyeight')

## make index for train, validation dataset
def make_split(X_data, n_splits, test_year):
  train_idxs=[]
  val_idxs=[]

  tss=BlockingTimeSeriesSplit(n_splits=n_splits)

  for train_idx, val_idx in tss.split(X_data[:-(test_year*12)]):
    train_idxs.append(train_idx)
    val_idxs.append(val_idx)

  return train_idxs, val_idxs

## plot the result of blocked split
def plot_cv_indices(cv, X, n_splits, lw=10):

    fig, ax = plt.subplots()

    for ii, (tr, tt) in enumerate(cv.split(X=X)):
        indices = np.array([np.nan] * len(X))
        indices[tt] = 1
        indices[tr] = 0

        ax.scatter(range(len(indices)), [ii + .5] * len(indices),
                   c=indices, marker='_', lw=lw, cmap=cmap_cv,
                   vmin=-.2, vmax=1.2)

    yticklabels = list(range(n_splits))
    ax.set(yticks=np.arange(n_splits) + .5, yticklabels=yticklabels,
           xlabel='Sample index', ylabel="CV iteration",
           ylim=[n_splits+0.1, -.1], xlim=[0, len(X)])
    ax.set_title('{}'.format(type(cv).__name__), fontsize=15)

    ax.legend([Patch(color=cmap_cv(.8)), Patch(color=cmap_cv(.02))],
          ['Testing set', 'Training set'], loc=(1.02, .8))

### Make LSTM/GRU model

* fitting with X, y where t is timestep length
-- X.shape=[#obs-(t-1), t, #feature]
-- y.shape=[#obs-(t-1), #class]
-- output.shape=[#obs-(t-1), #class]: ex.column 0 will show the predicted probability for class 0


In [ ]:
def make_lstm_model(num_layer, X_train, nc, metrics, loss_type, dropout, activation, regularizer):
    model = Sequential()
    model.add(LSTM(num_layer, input_shape=(X_train.shape[1], X_train.shape[2]), dropout=dropout, activation=activation, kernel_regularizer=regularizer))  ##input_shape=(X_train.shape[1], X_train.shape[2])
    model.add(Dense(nc, activation='softmax'))
    model.compile(loss=loss_type, optimizer='adam', metrics=metrics)
    return model

def make_gru_model(num_layer, X_train, nc, metrics, loss_type, dropout, activation, regularizer):
    model = Sequential()
    model.add(GRU(num_layer, input_shape=(X_train.shape[1], X_train.shape[2]), dropout=dropout, activation=activation, kernel_regularizer=regularizer))  ##input_shape=(X_train.shape[1], X_train.shape[2])
    model.add(Dense(nc, activation='softmax'))
    model.compile(loss=loss_type, optimizer='adam', metrics=metrics)
    return model

### Make Grid Search function which trains the model and automatically save the model and performance results on designated path

In [ ]:
######################run this cell only for training whole models####################################

## set the random seed for reproducibility
np.random.seed(699)

os.environ['PYTHONHASHSEED'] = '699'
random.seed(699)

tf.compat.v1.set_random_seed(699)
tf.keras.utils.set_random_seed(699)

session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
tf.compat.v1.keras.backend.set_session(sess)

def grid_search(path, X_data, y_data, model_type, y_type, n_splits, test_year, nl=[10, 20, 30, 40, 50, 70, 100],
                epochs=[100], batch_sizes=[6], time_steps=[12, 18, 24], es=True, dropouts=[0.2],  # batch_size=12
                activation='tanh', reg_methods=[None, 'L1','L1'], reg_factors=[0, 0.01, 0.03]):

## comment this part when the cell unexpected stop in the midst of grid-search
 results = []
  history_dict={'model':[], 'val':[], 'history':[],
                'auc': [],
                'recall_0': [], \
                'recall_1': [], \
                'recall_2': [], \
                'final_auc':[], \
                'final_recall_0':[],
                'final_recall_1':[],
                'final_recall_2':[],}

## uncomment this part when the cell unexpected stop in the midst of grid-search
  # with open(path+'/history_dict.pkl','rb') as f:
  #   history_dict=pickle.load(f)

  best_auc = 0
  best_params = None
  nc=len(y_data.unique())

  train_idxs, val_idxs = make_split(X_data, n_splits, test_year)

  for time_step in time_steps:
    loss_type = 'categorical_crossentropy'
    for num_epochs in epochs:
      for batch_size in batch_sizes:
        for num_layer in nl:
          for dropout in dropouts:
            for (method, factor) in zip(reg_methods, reg_factors):
              if method=='L1':
                reg=regularizers.L1(factor)
              elif method=='L2':
                reg=regularizers.L2(factor)
              else:
                reg=None

              if method==None:
                model_name='clf_{}_{}_{}_b{}_ep{}_h{}_d{}_{}'.format(y_type, model_type, time_step, batch_size, num_epochs, num_layer, dropout, method)
              else:
                model_name='clf_{}_{}_{}_b{}_ep{}_h{}_d{}_{}_{}'.format(y_type, model_type, time_step, batch_size, num_epochs, num_layer, dropout, method, factor)


              for i, (train_idx, val_idx) in enumerate(zip(train_idxs, val_idxs)):

                X_train=X_data.iloc[list(train_idx)]
                y_train=y_data.iloc[list(train_idx)]
                X_val=X_data.iloc[list(val_idx)]
                y_val=y_data.iloc[list(val_idx)]

                ##standard scaling
                scaler = StandardScaler()
                X_train = scaler.fit_transform(X_train)
                X_val = scaler.transform(X_val)

                ##make dataset to feed into the model (dataset split into timestep unit)
                trainx, trainy = create_dataset(X_train, y_train, time_step)
                valx, valy = create_dataset(X_val, y_val, time_step)
                trainy = to_categorical(trainy, num_classes=nc)
                valy = to_categorical(valy, num_classes=nc)

                ##assign metrics to calculate for each validation set
                if y_type!='y_agg':

                  metrics=[AUC(curve='ROC', multi_label=True, num_labels=2),  ##'Accuracy','Recall','Precision',
                          Recall(class_id=0, name='recall_0'),
                          Recall(class_id=1, name='recall_1')]

                else:

                  metrics=[AUC(curve='ROC', multi_label=True, num_labels=2),  ##'Accuracy','Recall','Precision',
                          Recall(class_id=0, name='recall_0'),
                          Recall(class_id=1, name='recall_1'),
                          Recall(class_id=2, name='recall_2')]

                # set model type
                if model_type=='lstm':
                  model=make_lstm_model(num_layer, trainx, nc, metrics, loss_type, dropout, activation=activation, regularizer=reg)
                else:
                  model=make_gru_model(num_layer, trainx, nc, metrics, loss_type, dropout, activation=activation, regularizer=reg)

                ## execute early stop according to recall for positive class
                if es==True:
                  early_stop = EarlyStopping(monitor='recall_1', patience=5, verbose=1,
                                            restore_best_weights=True)   ##restore best_weight=True
                  history=model.fit(trainx, trainy, epochs=num_epochs, batch_size=batch_size, callbacks=[early_stop])
                else:
                  history=model.fit(trainx, trainy, epochs=num_epochs, batch_size=batch_size)

                # evaluation of the model for each validation set
                scores = model.evaluate(valx, valy, verbose=0)

                # store the evaluation results in history_dict
                history_dict['model'].append(model_name)
                history_dict['val'].append(i)
                history_dict['history'].append(history)
                history_dict['auc'].append(scores[1])
                history_dict['recall_0'].append(scores[2])
                history_dict['recall_1'].append(scores[3])

                if y_type=='y_agg':
                  history_dict['recall_2'].append(scores[4])
                else:
                  history_dict['recall_2'].append(-100)

              X_test=X_data.iloc[-test_year*12-time_step+1:]   ##revising this part: -time_step+1
              y_test=y_data.iloc[-test_year*12-time_step+1:]   ##revising this part: -time_step+1

              X_ft=X_data.iloc[:-test_year*12]
              y_ft=y_data.iloc[:-test_year*12]

              ##standard scaling
              scaler = StandardScaler()
              X_ft = scaler.fit_transform(X_ft)
              X_test = scaler.transform(X_test)

              ##make dataset to feed into the model (dataset split into timestep unit)
              ft_x, ft_y = create_dataset(X_ft, y_ft, time_step)
              testx, testy = create_dataset(X_test, y_test, time_step)
              ft_y = to_categorical(ft_y, num_classes=nc)
              testy = to_categorical(testy, num_classes=nc)

              ##assign metrics to calculate for final test dataset
              if y_type!='y_agg':
                metrics=[AUC(curve='ROC', multi_label=True, num_labels=2),  ##'Accuracy','Recall','Precision',
                        Recall(class_id=0, name='recall_0'),
                        Recall(class_id=1, name='recall_1')]

              else:
                metrics=[AUC(curve='ROC', multi_label=True, num_labels=2),   ##'Accuracy','Recall','Precision',
                        Recall(class_id=0, name='recall_0'),
                        Recall(class_id=1, name='recall_1'),
                        Recall(class_id=2, name='recall_2')]

              ## set model type
              if model_type=='lstm':
                model=make_lstm_model(num_layer, ft_x, nc, metrics, loss_type, dropout, activation=activation, regularizer=reg)
              else:
                model=make_gru_model(num_layer, ft_x, nc, metrics, loss_type, dropout, activation=activation, regularizer=reg)

              ## execute early stop according to recall for positive class
              if es==True:
                early_stop = EarlyStopping(monitor='recall_1', patience=5, verbose=1,
                                          restore_best_weights=True)
                history=model.fit(ft_x, ft_y, epochs=num_epochs, batch_size=batch_size, callbacks=[early_stop])
              else:
                history=model.fit(ft_x, ft_y, epochs=num_epochs, batch_size=batch_size)

              ## evaluation of the model for final test set
              final_scores=model.evaluate(testx, testy, verbose=0)

              ## store the final test results in history_dict
              history_dict['final_auc'].extend([final_scores[1]]*n_splits)
              history_dict['final_recall_0'].extend([final_scores[2]]*n_splits)
              history_dict['final_recall_1'].extend([final_scores[3]]*n_splits)
              if y_type=='y_agg':
                history_dict['final_recall_2'].extend([final_scores[4]]*n_splits)
              else:
                history_dict['final_recall_2'].extend([-100]*n_splits)

              ## save validation and test results as history_dict.pkl file
              with open(path+'/history_dict.pkl','wb') as f:
                pickle.dump(history_dict, f)

              ## save trained model
              model.save(path+'/{}.h5'.format(model_name))




### Execute Grid Search
* Make directory for assigned target type, model type and early stopping options.  
* for each directory execute grid search and save the models as h5 format and results into history_dict.pkl file.

In [ ]:
######################run this cell only for training whole models####################################

########### assign target type, model type, and early stopping option(T/F) in interest ########################
### below example will return 4 folders with saved grid searched models and history_dict.pkl file ###
ytype_list=['oecd','oecd','oecd','oecd']
modeltype_list=['lstm', 'gru', 'lstm', 'gru']
es_list=[True, True, False, False]
###############################################################################################################

cases=list(zip(ytype_list, modeltype_list, es_list))

for (ytype, model_type, es) in cases:

  ## set random seed for environment
  np.random.seed(699)
  os.environ['PYTHONHASHSEED'] = '699'
  random.seed(699)

  tf.compat.v1.set_random_seed(699)
  tf.keras.utils.set_random_seed(699)

  session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
  sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
  tf.compat.v1.keras.backend.set_session(sess)

  ## assign threshold for minimum available years for features
  threshold=50

  y_type='y_{}'.format(ytype)
  n_splits=3
  test_year=5

  y=raw_df[y_type]
  y_data=y[-(threshold*12+6):-6]

  ## set folder path and generate directory if not exists
  if es==True:
    folder_path=path+'/model/change_test{}_{}_{}_es'.format(test_year, ytype, model_type)
  else:
    folder_path=path+'/model/change_test{}_{}_{}'.format(test_year, ytype, model_type)

  if not os.path.exists(folder_path):
    os.mkdir(folder_path)

  ## execute grid search
  ########### assign hyperparameters in interest #############################################################
  grid_search(folder_path, X_data, y_data, model_type, y_type, n_splits, test_year, nl=[10, 20, 30, 40, 50, 70, 100],
              epochs=[100], batch_sizes=[12], time_steps=[12, 18, 24], es=es, dropouts=[0.2],
              activation='tanh', reg_methods=[None, 'L1','L1'], reg_factors=[0, 0.01, 0.03])
  ############################################################################################################

Epoch 1/100
4/4 [==============================] - 3s 16ms/step - loss: 0.7385 - auc_57: 0.3438 - recall_0: 0.2857 - recall_1: 0.3857
Epoch 2/100
4/4 [==============================] - 0s 10ms/step - loss: 0.6872 - auc_57: 0.5789 - recall_0: 0.5397 - recall_1: 0.5714
Epoch 3/100
4/4 [==============================] - 0s 11ms/step - loss: 0.6610 - auc_57: 0.6406 - recall_0: 0.6032 - recall_1: 0.6286
Epoch 4/100
4/4 [==============================] - 0s 11ms/step - loss: 0.6416 - auc_57: 0.7025 - recall_0: 0.6349 - recall_1: 0.6429
Epoch 5/100
4/4 [==============================] - 0s 11ms/step - loss: 0.5950 - auc_57: 0.8108 - recall_0: 0.7619 - recall_1: 0.7429
Epoch 6/100
4/4 [==============================] - 0s 14ms/step - loss: 0.5730 - auc_57: 0.8694 - recall_0: 0.8095 - recall_1: 0.7857
Epoch 6: early stopping
Epoch 1/100
4/4 [==============================] - 3s 11ms/step - loss: 0.7166 - auc_58: 0.5213 - recall_0: 0.4396 - recall_1: 0.5714
Epoch 2/100
4/4 [=====================

## 2. Model Selection

## Concatenate results of all candidate models from all folders
* concatenate all results into df_result.pkl file
* group and average df_result.pkl by certain model specifications and early stopping options and save it to df_selection.pkl file

In [ ]:
folders=[x for x in os.listdir(path+"/model") if ('change' in x)]

for i, folder in enumerate(folders):
  folder_path=path+'/model/{}'.format(folder)
  with open(folder_path+'/history_dict.pkl','rb') as f:
    history_dict=pickle.load(f)

  ytype=folder.split("_")[2]
  model_type=folder.split("_")[3]
  if folder.split("_")[-1]=='es':
    es=True
  else:
    es=False

  df_tmp=pd.DataFrame(history_dict)
  df_tmp['ytype']=ytype
  df_tmp['model_type']=model_type
  df_tmp['early_stopping']=es

  if i==0:
    df_result=df_tmp
  else:
    df_result=pd.concat([df_result, df_tmp])

## model selection: according to cross validation recall, auc scores
df_result['macro_recall']=(df_result['recall_0']+df_result['recall_1'])/2
df_selection=df_result.groupby(['model','early_stopping']).mean().sort_values(['macro_recall'], ascending=[False])

with open(path+'/model/df_result.pkl', 'wb') as f:
  pickle.dump(df_result, f)

with open(path+'/model/df_selection.pkl', 'wb') as f:
  pickle.dump(df_selection, f)

<ipython-input-48-99fc569393a9>:27: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_selection=df_result.groupby(['model','early_stopping']).mean().sort_values(['macro_recall'], ascending=[False])


In [ ]:
with open(path+'/model/df_selection.pkl', 'rb') as f:
  df_selection = pickle.load(f)

model_name=df_selection.iloc[0].name[0]
es=df_selection.iloc[0].name[1]
print(model_name, es)

df_selection=df_selection.reset_index()
df_selection.head(10)

clf_y_oecd_lstm_24_b12_ep100_h10_d0.2_None True


,model,early_stopping,val,auc,recall_0,recall_1,recall_2,final_auc,final_recall_0,final_recall_1,final_recall_2,macro_recall
0,clf_y_oecd_lstm_24_b12_ep100_h10_d0.2_None,True,1.0,0.634259,0.666667,0.602564,-100.0,0.870536,0.732143,1.00,-100.0,0.634615
1,clf_y_oecd_gru_18_b12_ep100_h10_d0.2_None,True,1.0,0.433333,0.518519,0.680702,-100.0,0.642857,0.803571,0.25,-100.0,0.599610
2,clf_y_oecd_gru_18_b12_ep100_h10_d0.2_L1_0.01,True,1.0,0.585185,0.629630,0.563158,-100.0,0.886161,0.589286,1.00,-100.0,0.596394
3,clf_y_oecd_lstm_12_b12_ep100_h30_d0.2_None,False,1.0,0.447917,0.666667,0.463333,-100.0,0.888393,0.732143,1.00,-100.0,0.565000
4,clf_y_oecd_gru_12_b12_ep100_h30_d0.2_None,True,1.0,0.464120,0.629630,0.476667,-100.0,0.863839,0.821429,0.75,-100.0,0.553148
5,clf_y_oecd_lstm_12_b12_ep100_h50_d0.2_L1_0.03,True,1.0,0.519676,0.666667,0.437500,-100.0,0.957589,0.696429,1.00,-100.0,0.552083
6,clf_y_oecd_lstm_18_b12_ep100_h70_d0.2_None,True,1.0,0.420370,0.666667,0.428070,-100.0,0.908482,0.714286,1.00,-100.0,0.547368
7,clf_y_oecd_lstm_12_b12_ep100_h20_d0.2_None,False,1.0,0.483796,0.666667,0.415833,-100.0,0.852679,0.785714,0.75,-100.0,0.541250
8,clf_y_oecd_gru_12_b12_ep100_h30_d0.2_None,False,1.0,0.431713,0.666667,0.410000,-100.0,0.872768,0.589286,1.00,-100.0,0.538333
9,clf_y_oecd_gru_12_b12_ep100_h20_d0.2_None,True,1.0,0.439815,0.592593,0.480833,-100.0,0.866072,0.625000,0.75,-100.0,0.536713
